In [10]:
using Pkg; Pkg.activate(".")
using CalibrateEmulateSample.Utilities
using CalibrateEmulateSample.Emulators
import EnsembleKalmanProcesses as EKP
import ClimaCalibrate as CAL
import JLD2
import YAML

  Activating project at `/central/home/jschmitt/ClimaAtmos.jl/calibration/experiments/local_calibration/CES`


In [27]:
tot_iter = 3
output_dir = joinpath("../output", "local_calibration")
eki_filepath = joinpath(CAL.path_to_iteration(output_dir, tot_iter), "eki_file.jld2")
ekiobj = JLD2.load_object(eki_filepath)

# load eki object into CES input output pairs
input_output_pairs = Utilities.get_training_points(ekiobj, 2)
input_output_pairs

EnsembleKalmanProcesses.DataContainers.PairedDataContainer{Float64}(EnsembleKalmanProcesses.DataContainers.DataContainer{Float64}([-1.556040664338548 -2.053457352301882 … -2.028468589899718 -2.3017550973692456; 0.17708017774064289 0.14128754922443715 … 0.08172468753780245 0.11438165406231426; … ; -6.8029787057653195 -7.2139937730169645 … -7.217232974615285 -6.904965843070573; -1.5088665423578147 -1.726934462691475 … -1.4243889641107712 -1.2662776884890246]), EnsembleKalmanProcesses.DataContainers.DataContainer{Float64}([405.7225646972656 405.72564697265625 … 405.7229309082031 405.7237548828125; 0.3606768250465393 0.3608790636062622 … 0.3605056703090668 0.36057189106941223; … ; 274.0641174316406 275.0635681152344 … 274.0350646972656 274.0800476074219; 123.0937728881836 123.09915161132812 … 123.09416198730469 123.09424591064453]))

In [21]:
# get obs_noise_cov 
obs_noise_cov = JLD2.load_object("../obs_noise_cov.jld2")

9×9 LinearAlgebra.Diagonal{Float64, Vector{Float64}}:
 0.3   ⋅      ⋅        ⋅    ⋅       ⋅    ⋅    ⋅    ⋅ 
  ⋅   0.036   ⋅        ⋅    ⋅       ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅     0.00042   ⋅    ⋅       ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅      ⋅       5.0   ⋅       ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅      ⋅        ⋅   5.0e-5   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅      ⋅        ⋅    ⋅      1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅      ⋅        ⋅    ⋅       ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅      ⋅        ⋅    ⋅       ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅      ⋅        ⋅    ⋅       ⋅    ⋅    ⋅   1.0

In [28]:
# build and train emulator 
gppackage = Emulators.SKLJL()
gauss_proc = GaussianProcess(gppackage)

emulator = Emulator(gauss_proc, input_output_pairs,
                    obs_noise_cov = obs_noise_cov,
                    retained_svd_frac = .95,
)
optimize_hyperparameters!(emulator)

SVD truncated at k: 5/9
Using default squared exponential kernel, learning length scale and variance parameters
Using default squared exponential kernel:PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1])
Learning additive white noise
Tuple{String, String, Matrix{Float64}, Int64, Bool}[("k1__k1__constant_value", "numeric", [1.0e-5 10000.0], 1, 0), ("k1__k2__length_scale", "numeric", [1.0e-5 10000.0; 1.0e-5 10000.0; 1.0e-5 10000.0; 1.0e-5 10000.0; 1.0e-5 10000.0], 5, 0), ("k2__noise_level", "numeric", [1.0e-5 10.0], 1, 0)]
Completed training of: 1, 
PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1)


/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/

2, 
PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1)
3

/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(

, 
PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1)


/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit agai

4, 
PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1)
5

/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(

, 
PyObject 1**2 * RBF(length_scale=[1, 1, 1, 1, 1]) + WhiteKernel(noise_level=1)
SKlearn, already trained. continuing...


/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/jschmitt/.julia/conda/3/x86_64/lib/python3.10/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 2 of parameter k1__k2__length_scale is close to the specified upper bound 10000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/

In [24]:
# train emulator
optimize_hyperparameters!(emulator)


SKlearn, already trained. continuing...
